<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2023/ex03notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2023 ex03notebookA

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2023

----
## 重回帰分析 (1)
----

これまでの回帰分析では，ゴリゴリ君の例（「気温」対「アイス売上数」）のように説明変数も被説明変数も1つの変数だけでできている場合を考えていました．
ここでは，説明変数が2つ以上の変数でできている場合の回帰分析について考えます．

どちらも **回帰分析** (regression analysis) ですが，区別をする場合は，前者を **単回帰分析**(simple regression analysis)，後者を **重回帰分析**(multiple regression analysis) と呼びます．


以下，コードセルを上から順に実行しながら読んでいってね．

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
seaborn.set()

---
### やりたいこと

具体的なデータを例にあげて，重回帰分析でやりたいことを説明します．

次のセルを実行すると，データを Pandas の DataFrame として読み込みます．
ここでは Pandas/DataFrame についての理解は求めませんので，このコードセルが何をやっているかはわからなくても構いません．

ここで読み込んでいるデータは，

「Pythonで理解する統計解析の基礎」 谷合廣紀，辻 真吾，技術評論社，2018.

に掲載されているものです．以下の GitHub サイトで公開されています．
https://github.com/ghmagazine/python_stat_sample

In [ ]:
# データの読み込み
df = pd.read_csv('https://github.com/ghmagazine/python_stat_sample/raw/master/data/ch12_scores_reg.csv')
df.drop(columns='通学方法', inplace=True)
df

このデータは，とある授業の受講者20名について，「小テスト」と「期末テスト」の点数，および各受講者の「期末テスト」前日の「睡眠時間」という3つのデータを集めたものとなっています（元のデータにはこれ以外に「通学方法」というのもありますが，ここでは省いています）．

このようなデータを見たときに，「小テスト」と「睡眠時間」が「期末テスト」に影響していると考えるのは自然なことでしょう．
単回帰分析を使ってその影響を調べるとしたら，次のような二通りのやり方が考えられます：

- 「小テスト」を説明変数とし「期末テスト」を被説明変数とする
- 「睡眠時間」を説明変数とし「期末テスト」を被説明変数とする

以下のセルで実際にそのような単回帰分析をやってみています．

In [ ]:
## 「小テスト」 vs 「期末テスト」の単回帰分析
xvec = df['小テスト'].to_numpy()
yvec = df['期末テスト'].to_numpy()
X1 = np.vstack([xvec, np.ones_like(xvec)]).T
#print(X1.shape, yvec.shape)
rv = np.linalg.lstsq(X1, yvec, rcond=None)
a1, b1 = rv[0]
print(f'a1 = {a1:.3f}, b1 = {b1:.2f}')

In [ ]:
## 「睡眠時間」 vs 「期末テスト」の単回帰分析
xvec = df['睡眠時間'].to_numpy()
yvec = df['期末テスト'].to_numpy()
X1 = np.vstack([xvec, np.ones_like(xvec)]).T
#print(X1.shape, yvec.shape)
rv = np.linalg.lstsq(X1, yvec, rcond=None)
a2, b2 = rv[0]
print(f'a2 = {a2:.3f}, b2 = {b2:.2f}')

In [ ]:
## 二つの単回帰分析の結果を並べて可視化
fig, ax = plt.subplots(1, 2, figsize=(12,5))
ax[0].scatter(df['小テスト'], df['期末テスト'])
xx = np.array([0, 10])
ax[0].plot(xx, a1*xx + b1, color='red', label=f'$y = {a1:.3f} x + {b1:.2f}$')
ax[0].set_xlim(0, 10)
ax[0].set_ylim(0, 100)
ax[0].set_xlabel('Quiz')
ax[0].set_ylabel('Exam')
ax[0].legend()
ax[1].scatter(df['睡眠時間'], df['期末テスト'])
ax[1].plot(xx, a2*xx + b2, color='red', label=f'$y = {a2:.3f} x + {b2:.2f}$')
ax[1].set_xlim(0, 10)
ax[1].set_ylim(0, 100)
ax[1].set_xlabel('Sleep')
ax[1].set_ylabel('Exam')
ax[1].legend()
plt.show()

このように，単回帰分析を使えば「小テスト」が「期末テスト」に及ぼす影響や「睡眠時間」が「期末テスト」に及ぼす影響を調べることはできます．
しかし，説明変数は「小テスト」と「睡眠時間」のどちらか一方だけですので，両方あわせたものが「期末テスト」にどんな影響を及ぼすのかはよくわかりません．



このような場合に重回帰分析を使うと，「小テスト」と「睡眠時間」の両方を説明変数とし「期末テスト」を被説明変数とした回帰分析を行うことができます．

具体的には，次のような式を考えます．
$$
y = w_0 + w_1x_1 + w_2x_2 \qquad (*)
$$
ここで，$x_1, x_2, y$ はそれぞれ，「小テスト」，「睡眠時間」，「期末テスト」の値を表します．
$w_0, w_1, w_2$ の値を適当に決めれば，$(x_1, x_2)$ から $y$ の値を求めることができます．
したがって，単回帰分析の場合と同じように，与えられたデータにうまく当てはまるように $(w_0, w_1, w_2)$ の値を決めてやろう，という問題設定を考えることができます．

ちゃんとした話（「うまく当てはまる」のちゃんとした定義とか）は後回しにして，とりあえず上記のデータの例でやってみましょう．まず，$(x_1, x_2, y)$ の散布図を描いてみるとこんなんです．

In [ ]:
# (小テスト, 睡眠時間, 期末テスト) の散布図
fig = plt.figure(figsize=(10, 7.5))
ax = fig.add_subplot(projection='3d')
ax.scatter(df['小テスト'], df['睡眠時間'], df['期末テスト'], s=50, color='blue')
ax.set_xlabel('$x_1$: Quiz')
ax.set_xlim(0, 10)
ax.set_ylabel('$x_2$: Sleep')
ax.set_ylim(0, 10)
ax.set_zlabel('$y$: Exam')
ax.set_zlim(0, 100)
ax.view_init(25, -70)
plt.show()

個々の青い点がひとりひとりの $(小テスト, 睡眠時間, 期末テスト)$　の値を表しています．
これに対して，式$(*)$ はこの空間に広がる2次元平面を表します．
$(w_0, w_1, w_2)$ の値をいろいろ変えると平面の向きや位置が変わりますが，（最小二乗法を用いて）データに最もよく当てはまる平面の式を求めて表示すると，次のようになります．

In [ ]:
# (小テスト, 睡眠時間) vs 期末テストの重回帰分析
X = df.loc[:, ['小テスト', '睡眠時間']].to_numpy()
Y = df['期末テスト'].to_numpy()
X1 = np.vstack([np.ones(len(X)), X.T]).T
rv = np.linalg.lstsq(X1, Y, rcond=None)
w = rv[0]
print(f'w_0 = {w[0]:.3f},   w_1 = {w[1]:.3f},   w_2 = {w[2]:.3f}')

# 平面を可視化するためのデータの準備
xmin, xmax = 0, 10
ymin, ymax = 0, 10
x_mesh, y_mesh = np.meshgrid(np.linspace(xmin, xmax, num=16), np.linspace(ymin, ymax, num=16))
X1mesh = np.vstack((np.ones(len(x_mesh.ravel())), x_mesh.ravel(), y_mesh.ravel())).T
z_mesh = (X1mesh @ w).reshape((x_mesh.shape[0], y_mesh.shape[1]))

# 散布図と得られた平面を重ねて描画
fig = plt.figure(figsize=(10, 7.5))
ax = fig.add_subplot(projection='3d')
ax.scatter(X1[:, 1], X1[:, 2], Y, s=50, color='blue')
ax.plot_wireframe(x_mesh, y_mesh, z_mesh, color='red',
                  label=f'$y = {w[0]:.2f} + {w[1]:.2f} x_1 + {w[2]:.2f} x_2$')
ax.set_xlabel('$x_1$: Quiz')
ax.set_xlim(0, 10)
ax.set_ylabel('$x_2$: Sleep')
ax.set_ylim(0, 10)
ax.set_zlabel('$y$: Exam')
ax.set_zlim(0, 100)
ax.view_init(25, -70)
ax.legend()
plt.show()

単回帰で得られた係数とこちらで得られた係数 $(w_0, w_1, w_2)$ の値は異なっていることに注意してください．

このように式が得られれば，単回帰分析の場合と同じようにして，適当な $(x_1, x_2)$ の値に対する $y$ の値の予測値を算出することができますね．

In [ ]:
# (1, Quiz, Sleep) をならべた 9 x 3 行列
XX = np.array([
    [1, 2, 2], [1, 5, 2], [1, 8, 2],
    [1, 2, 5], [1, 5, 5], [1, 8, 5],
    [1, 2, 8], [1, 5, 8], [1, 8, 8],
])
# Exam の予測値
YY = XX @ w
# 表示
for n in range(len(YY)):
    print(f'小テスト {XX[n, 1]} 点，前日の睡眠時間 {XX[n, 2]} 時間のひとの期末テストの点数の予測値は {YY[n]:.1f} 点')

---
### 問題の定式化

重回帰分析の問題をきちんと定式化しましょう（注）．

<span style="font-size: 75%">
※注: ここでは確率統計の言葉を用いた定式化をしていませんので，重回帰分析の問題の定式としては完全ではありません．その辺のことはまた後日解説します．
</span>


---
$D$個の値 $x_1, x_2, \ldots, x_D$ から一つの値 $y$ が予測できるとして，それらの間に

$$
y = w_0 + w_1x_1 + w_2x_2 + \cdots + w_Dx_D\qquad (1)
$$

という関係が成り立つと仮定します． $x_1, x_2, \ldots, x_D$ を **説明変数**， $y$ を **被説明変数** といいます（注）．

$(x_1, \ldots, x_D)$ と $y$ の組 $N$ 個から成るデータ

$$
\begin{array}{c}
((x_{1,1}, x_{1,2}, \ldots, x_{1,D}), y_1),\\
((x_{2,1}, x_{2,2}, \ldots, x_{2,D}), y_2),\\
\vdots \\
((x_{N,1}, x_{N,2}, \ldots, x_{N,D}), y_N)\\
\end{array}
$$

が与えられたとき，式(1)がこれらのデータになるべく「良く当てはまる」ように $(D+1)$ 個のパラメータ $w_0, w_1, w_2, \ldots, w_D$ の値を決めましょう．

---

<span style="font-size: 75%">
※注: 説明変数を「独立変数」と呼ぶこともあります．被説明変数を「従属変数」または「目的変数」と呼ぶこともあります．
</span>

これが重回帰分析の問題です．
式(1)は，説明変数と被説明変数が作る $(D+1)$ 次元空間中の $D$次元平面を表す式です．上のデータの例では，$D=2$ で，$(小テスト, 睡眠時間, 期末テスト)$ の3次元空間中の2次元平面を表しています．

重回帰分析の目的は，この平面がデータの点たちに「良く当てはまる」ようにパラメータを決めることです．ただし，この表現では「良く当てはまる」という部分がまだあいまいですので，ここをきちんと定めましょう．


この問題において，$(x_{n,1}, x_{n,2}, \ldots, x_{n,D})$ を式(1)に代入して得られる値

$$
\hat{y}_n = w_0 + w_1x_{n,1} + w_2x_{n,2} + \cdots + w_Dx_{n,D}
$$

は， $y$ の予測値といえます．
一方，$(x_{n,1}, x_{n,2}, \ldots, x_{n,D})$ に対応して実際に得られた $y$ の値は $y_n$ です．実際に得られた値 $y_n$ と予測値 $\hat{y}_n$ とのずれを二乗誤差
$
(y_n - \hat{y}_n)^2
$
で測ることにして，$N$個のデータについての二乗誤差の和（**残差平方和**）

$$
\begin{aligned}
E(w_0, w_1, \ldots, w_D) &= \sum_{n=1}^{N}(y_n - \hat{y}_n)^2 \\
&= \sum_{n=1}^{N}(y_n - (w_0 + w_1x_{n,1} + w_2x_{n,2} + \cdots + w_Dx_{n,D}))^2
\end{aligned}
$$

を考えてみます．
予測値と実際の値とのずれが小さければ小さいほど，この値は小さくなりますから，この値は，データに対する式(1)の当てはまりの「良さ」の指標とみなせます（値が小さい方が良いので，より正確には「悪さ」の指標）．

これで「良く当てはまる」の定義ができました．
これを使って問題設定の最後の部分を書き直すと，次のようになります．


---

これらのデータに対する式(1)の当てはまりの良さを残差平方和

$$
E(w_0, w_1, \ldots, w_D) = \sum_{n=1}^{N}(y_n - (w_0 + w_1x_{n,1} + w_2x_{n,2} + \cdots + w_Dx_{n,D}))^2 \qquad (2)
$$

で測るものとして，これが最小となるように  $(D+1)$ 個のパラメータ $w_0, w_1, w_2, \ldots, w_D$ の値を決めましょう．

---

ここでは，実際の値と予測値との二乗誤差の和を最小にする問題という形で定式化しました．
これが最も一般的な重回帰分析ですが，当てはまりの良さの規準に二乗誤差とは異なるものを用いる場合もあります．特に区別したい場合は，ここで説明している方法を「**最小二乗法**による重回帰分析」と呼びます（注）．


<span style="font-size: 75%">
※注: データの性質や分析の目的によっては，二乗誤差とは異なる規準を用いた方が良い場合があります（発展的な内容になりますのでこれ以上の説明は省略します）．
</span>


---
### 最小二乗法の解

最小二乗法による重回帰分析の問題の解は，単回帰の場合と同様の手順で求めることができます．
ただし，文字が多くて式が複雑になるので，ベクトルを使って式を整理して考えることにします．

まず，$n$番目のデータの値 $x_{n,1}, x_{n,2}, \ldots, x_{n,D}$ をならべたものの先頭に $1$ を付け足した $(D+1)$ 次元ベクトルを

$$
\mathbf{x}_n = (1, x_{n,1}, x_{n,2}, \ldots, x_{n,D}) \qquad (n = 1, 2, \ldots, N)
$$

と表すことにします．先頭に余分な $1$ が付いているため， $D$ 次元ではなく $(D+1)$ 次元になっていることに注意．

これに対して，$(D+1)$ 個のパラメータ $w_0, w_1, \ldots, w_D$ をならべた $(D+1)$ 次元ベクトルを

$$
\mathbf{w} = (w_0, w_{1}, w_{2}, \ldots, w_{D})
$$

と表すことにすると， $\hat{y}_n$ を簡単に $\hat{y}_n = \mathbf{w}\cdot \mathbf{x}_n$ と書くことができます（$\mathbf{x}\cdot\mathbf{y}$ はベクトル $\mathbf{x}$ とベクトル $\mathbf{y}$ の内積）．
最小化したい残差平方和は

$$
E(\mathbf{w}) = \sum_{n=1}^{N}(y_n - \hat{y}_n)^2 = \sum_{n=1}^{N}(y_n - \mathbf{w}\cdot\mathbf{x}_n)^2
$$

です．

準備ができました．
われわれの目的は，$E(\mathbf{w})$ を最小にする $\mathbf{w}$ を求めることです．
単回帰の解の導出過程と同じように，$E(\mathbf{w})$ を $\mathbf{w}$ の各要素で偏微分したものを $0$ とおいて，解が満たす方程式を求めます．
$E(\mathbf{w})$ の $w_d$ ($d=0,1,\ldots,D$) での偏微分は次のように計算できます．

$$
\begin{aligned}
\frac{\partial E(\mathbf{w})}{\partial w_d} &= 2\sum_{n=1}^N (y_n - \mathbf{w}\cdot\mathbf{x}_n) \frac{\partial}{\partial w_d}(y_n - \mathbf{w}\cdot\mathbf{x}_n)   \\
&= 2\sum_{n=1}^N (y_n - \mathbf{w}\cdot\mathbf{x}_n)\left( 0 -\frac{\partial}{\partial w_d}(w_0+w_1x_{n,1}+\cdots + w_Dx_{n,D})\right) \\
&= 2\sum_{n=1}^N (y_n - \mathbf{w}\cdot\mathbf{x}_n)(-x_{n,d}) \qquad (d = 0, 1, 2, \ldots, D)
\end{aligned}
$$

$\mathbf{x}_n$ も $\mathbf{w}$ も要素は添字の番号が $0$ のものから $D$ のものまである（$(D+1)$個ある）ことに注意してください．さらに，$\mathbf{x}_n$ の $0$ 番目の要素 $x_{n,0}$ は常に $1$ です．

$\frac{\partial E(\mathbf{w})}{\partial w_d} = 0$ とおくと， $(D+1)$ 個の未知数 $w_0, w_1, \ldots, w_D$ に対して $(D+1)$ 個の式から成る次の連立方程式が得られます．

$$
\left\{
    \begin{aligned}
    \sum x_{n,0}\mathbf{w}\cdot\mathbf{x}_n &= \sum x_{n,0}y_n\\
    \sum x_{n,1}\mathbf{w}\cdot\mathbf{x}_n &= \sum x_{n,1}y_n\\
    \vdots \\
    \sum x_{n,D}\mathbf{w}\cdot\mathbf{x}_n &= \sum x_{n,D}y_n\\
    \end{aligned}
\right. \qquad (3)
$$

簡単のため，和記号 $\sum$ の上下の $n=1$ と $N$ を省略して書いてます．


この連立方程式を $\mathbf{w}$ について解けば残差平方和を最小にするパラメータ $\mathbf{w}$ が求まりますが，このままでは見通しが悪いので，単回帰の場合と同様にデータの値をならべた行列を考えて式を整理することにします．

まず，ベクトル $\mathbf{x}_n$ を行ベクトル（要素が行方向に並んだベクトル）とみなして，それらを列方向に並べた行列を $X$ とします．

$$
X = \begin{pmatrix}
\mathbf{x}_1\\
\mathbf{x}_2\\
\vdots\\
\mathbf{x}_N\\
\end{pmatrix}
=
\begin{pmatrix}
1 & x_{1,1} & x_{1,2} & \cdots & x_{1,D}\\
1 & x_{2,1} & x_{2,2} & \cdots & x_{2,D}\\
& & \vdots\\
1 & x_{N,1} & x_{N,2} & \cdots & x_{N,D}\\
\end{pmatrix}
$$

$X$ は $N\times(D+1)$ の行列です．

次に，$y_1, y_2, \ldots, y_N$ を列方向に並べた行列を $Y$ とします．

$$
Y = \begin{pmatrix}
y_1\\ y_2 \\ \vdots \\ y_N
\end{pmatrix}
$$

$Y$ は $N\times 1$ の行列です．

このとき，式(3)の連立方程式は，次のように書き直すことができます（注）．

$$
X^{\top}X
\begin{pmatrix}
w_0\\ w_1\\ \vdots \\ w_D
\end{pmatrix}
= X^{\top}Y \qquad (4)
$$

この連立方程式は，単回帰分析と同様に **正規方程式** と呼ばれます．
$D=1$ とすれば単回帰の正規方程式と等しいことが分かります（1を付け加えている場所が違うのでパラメータの並び順が違うだけ）．
この正規方程式の解が，$E(\mathbf{w})$ を最小にする $\mathbf{w}$ となります．






※注: 式変形の過程を少し省略しました．省略した部分を以下に書いておきます．
式(3) の左辺の $d$ 番目（$d=0,1,\ldots,D$）の行を取り出して考えると，

$$
\begin{aligned}
\sum_{n=1}^{N} x_{n,d}\mathbf{w}\cdot\mathbf{x}_n &= \sum_{n=1}^{N} (x_{n,d}\mathbf{x}_n)\cdot \mathbf{w} = \left( \sum_{n=1}^{N} x_{n,d}\mathbf{x}_n\right) \cdot\mathbf{w} \\
&= \left( \sum x_{n,d}x_{n,0}, \sum x_{n,d}x_{n,1},  \ldots, \sum x_{n,d}x_{n,D} \right) \cdot \mathbf{w}
\end{aligned}
$$

と書けるので，式(3)の左辺は

$$
\begin{pmatrix}
\sum x_{n,0}x_{n,0} & \sum x_{n,0}x_{n,1} & \cdots & \sum x_{n,0}x_{n,D} \\
\sum x_{n,1}x_{n,0} & \sum x_{n,1}x_{n,1} & \cdots & \sum x_{n,1}x_{n,D} \\
\vdots & \vdots & & \vdots\\
\sum x_{n,D}x_{n,0} & \sum x_{n,D}x_{n,1} & \cdots & \sum x_{n,D}x_{n,D} \\
\end{pmatrix}
\begin{pmatrix}
w_0 \\ w_1 \\ \vdots \\ w_D
\end{pmatrix}
= \left(\sum_{n=1}^{N} \mathbf{x}_n^{\top} \mathbf{x}_n \right)
\begin{pmatrix}
w_0 \\ w_1 \\ \vdots \\ w_D
\end{pmatrix}
= X^{\top}X
\begin{pmatrix}
w_0 \\ w_1 \\ \vdots \\ w_D
\end{pmatrix}
$$

となります．

式(3)の右辺の方はもっと簡単ですので，自分で導出してみてね．

---
### 決定係数




回帰分析の結果の良し悪しを表す指標の一つに，**決定係数** というものがあります（「寄与率」と呼ぶこともあります）．
**決定係数** は，「被説明変数の変動のうちどれくらいを説明変数によって説明できているか」を表す値です． 多くの場合，$r^2$ または $R^2$ という記号で表され，次のように定義されます．

$$
r^2 = 1 - \frac{\displaystyle\sum_{n=1}^{N}(y_n - \hat{y}_n)^2}{\displaystyle\sum_{n=1}^{N}(y_n - \bar{y})^2}
$$

ただし，$\hat{y}_n$ は上で説明している通り$(x_{n,1}, x_{n,2}, \ldots, x_{n,D})$ に対応する予測値であり，$\bar{y}$ は $y$ の平均です．

分子の式

$$\sum_{n=1}^{N}(y_n - \hat{y}_n)^2$$

は，「予測残差の平方和」です．
式(1)で $(x_{n,1}, x_{n,2}, \ldots, x_{n,D})$ から $y$ を予測してみたが予測しきれず残った値の散らばり，を表します．
一方，分母の式

$$\sum_{n=1}^{N}(y_n - \bar{y})^2$$

は，$y$ の分散を $N$ 倍したものになっており，「被説明変数$y$の変動の大きさ」，つまり，もともと $y$ の値がどれくらい散らばっていたか，を表します．つまり，

$$
r^2 = 1 - \frac{\mbox{予測しきれず残った値の散らばりの大きさ}}{\mbox{もともとの$y$の散らばりの大きさ}}
$$

ということです．

説明変数から被説明変数が完全に予測できた場合は，残差平方和が $0$ になりますので，$r^2 = 1$ となります．
一方，予測が全くうまくいかない場合は，もともとの $y$ の散らばりが全部残ったままで分子の残差平方和が分母の値と等しくなるので，$r^2 = 0$ となります．
したがって，$0 \leq r^2 \leq 1$ であり，$r^2$ の値は $1$ に近いほど当てはまりがよい，ということになります．

しかし，実は，決定係数は，重回帰分析の結果の良し悪しを判断する指標としてはあまり良いものではありません．
次回，その理由と，その点を改良した指標について説明します．




以下の図は，上記の説明を直感的に理解するための図です（ただし，重回帰分析ではなく単回帰分析の例です）．
上の段が決定係数が大きい場合，下の段が決定係数が小さい場合で，
左側が「もともとの $y$ の散らばりの大きさ」を，右側が「予測しきれず残った値の散らばりの大きさ」を表しています．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/Data/ex11-residual.png">

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/Data/ex11-residual2.png">